In [1]:
from gensim.models import KeyedVectors
import numpy as np
from joblib import dump, load
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [2]:
# Usa frecuencia de palabras
def create_tf_idf(list_text):
    vectorizer = TfidfVectorizer(use_idf=False, norm=None, ngram_range=(1, 1)) # use_idf=true ponderacion de frecuancia de palabras en documentos
    tfidf = vectorizer.fit(list(list_text))
    vectores_tfidf=vectorizer.transform(list_text)
    dump(tfidf,'tfidf_model.joblib')
    print('Fue creado modelo TF-IDF')
    #vectores_tfidf=list(matrix)
    #print(len(vectores_tfidf))
    return vectores_tfidf

In [3]:
# Retorna un vector TF-IDF 
def get_vector_tfidf(list_text):
    tfidf=load('tfidf_model.joblib')# Ojo cuidado, solo cargar solo una vez
    words_ntf=[x for x in list_text.split() if x not in tfidf.get_feature_names()] # Palabras que no estan en el vocabulario del modelo
    return tfidf.transform([list_text]).toarray()

In [4]:
# Leyendo los datos CSV
datos_notacion = pd.read_csv('natacion_suicidio.csv')
datos_general = pd.read_csv('suicidio_todo.csv') 

In [5]:
# seleccionamos solo los datos que tienen tendencia suicida
selec_dat_1=datos_notacion.loc[datos_notacion['suicidio'] == 1]
selec_dat_1.head(2)

,frase,tweet_clean,suicidio
7,cansado de estar solo,louis espero no estar pidiendo mucho solo quie...,1
17,cansado de estar solo,estoy super cansado cansado de la gente cansad...,1


In [6]:
datos_general.head(2)

,id,frase,idtw,tweet,nombre,fecha,tweet_clean
0,4901,dormir y nunca mas despertar,1185025213177704448,Ganas de dormir y no despertar nunca más,darafiorodri,18/10/2019 02:50,ganas de dormir y no despertar nunca más
1,4902,dormir y nunca mas despertar,1185004540707430400,RT @MarioMcmv75: Así es NUNCA MÁS NOS TRAGAMOS...,sanpascualitor,18/10/2019 01:27,así es nunca más nos tragamos y cerquita de di...


In [7]:
# Agrupamos y guardamos en un diccionario, cada elemento del diccionario es una lista
dict_general=datos_general.groupby('frase')['tweet_clean'].apply(list).to_dict()

# Extraemos todo los tweets para crear el vocabulario para TF-IDF
text_todo=list(datos_general.tweet_clean.values.astype('U'))
vectores_tfidf=create_tf_idf(text_todo) #vocubulario total

Fue creado modelo TF-IDF


In [ ]:
similares=defaultdict(list) #inicializamos un diccionario de listas para guardar la data necesaria
for i,j in zip(list(selec_dat_1.frase),list(selec_dat_1.tweet_clean)):
    vec_selec=get_vector_tfidf(j)
    print(i)
    for x in dict_general[i]:
        vec_todo_j=get_vector_tfidf(x)
        simi=cosine_similarity(vec_selec,vec_todo_j)[0][0]
        if simi>0.6 and simi<0.95: # guardamos todos los tweets con similaridad alta, pero menor 0.95 que podria ser la misma frase
            similares['frase'].append(i)
            similares['tweet_anotado'].append(j)
            similares['tweet_similar'].append(x)
            similares['similaridad'].append(simi)
            

cansado de estar solo
cansado de estar solo


In [ ]:
# convertimos el diccionario de listas a un Dataframe e lo guardamos como CSV
findal_dt=pd.DataFrame.from_dict(similares,orient='index').transpose()
findal_dt.to_csv(r'datos_similares_notados.csv')